# Getting the Judges

[Source](http://www.bvger.ch/gericht/richter/00563/00580/index.html?lang=de) of the judges.
- [Abteilung IV](http://www.bvger.ch/gericht/richter/00563/00580/index.html?lang=de)
- [Abteilung V](http://www.bvger.ch/gericht/richter/00563/00581/index.html?lang=de)

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

plt.style.use('ggplot')
import dateutil.parser
import re
import time

In [3]:
url = "http://www.bvger.ch/gericht/richter/00563/00580/index.html?lang=de"
response = requests.get(url)
judges_IV_soup = BeautifulSoup(response.text, 'html.parser')
judges_IV = judges_IV_soup.find_all('div', {'class': 'contentFlex flexUnterseite'})

In [4]:
judges_IV_list = []

for soup in judges_IV:
    
    #name
    name = soup.find('a').text
    name = name.replace(' (PrÃ¤sident Abt. IV)', '')
    
    #url
    html = soup.find('a').get('href')
    url = 'http://www.bvger.ch' + html
    
    response = requests.get(url)
    judges_text = BeautifulSoup(response.text, 'html.parser')
    partei = judges_text.find('div', {'class': 'webText flexTinymceDiv'}).text[-3:]
    partei = partei.replace('los', 'parteilos').replace('ux.', 'FDP').replace('PLR', 'FDP')
    partei = partei.replace('que', 'parteilos').replace('üne', 'Grüne')
    partei = partei.replace('UDC', 'SVP').replace('DC.', 'SVP').replace('DC.', 'FDP')
    
    judge = {'Name': name,
             'Partei': partei,
            }
    
    judges_IV_list.append(judge)

In [5]:
url = "http://www.bvger.ch/gericht/richter/00563/00581/index.html?lang=de"
response = requests.get(url)
judges_V_soup = BeautifulSoup(response.text, 'html.parser')
judges_V = judges_V_soup.find_all('div', {'class': 'contentFlex flexUnterseite'})

In [6]:
judges_V_list = []

for soup in judges_V:
    
    #name
    name = soup.find('a').text
    name = name.replace(' (PrÃ¤sident Abt. IV)', '').replace(' (PrÃ¤sidentin Abt. V)', '')
     
    #url
    html = soup.find('a').get('href')
    url = 'http://www.bvger.ch' + html
    
    response = requests.get(url)
    judges_text = BeautifulSoup(response.text, 'html.parser')
    partei = judges_text.find('div', {'class': 'webText flexTinymceDiv'}).text[-3:]
    partei = partei.replace('los', 'parteilos').replace('ux.', 'FDP')
    partei = partei.replace('que', 'parteilos').replace('rts', 'Grüne').replace('üne', 'Grüne').replace('GrGrüne', 'Grüne')
    partei = partei.replace('UDC', 'SVP').replace('DC.', 'SVP').replace('DC.', 'FDP')
    partei = partei.replace('07.', 'parteilos').replace('ale', 'GLP').strip()
    partei = partei.replace('PS', 'SP').replace('VP.', 'SVP')
    
    
    judge = {'Name': name,
             'Partei': partei,
            }
    
    judges_V_list.append(judge)

In [7]:
judges_list = judges_IV_list + judges_V_list

In [8]:
judges_list = pd.DataFrame(judges_list)

# Reorganising the Names to merge later on

In [9]:
test = list(judges_list['Name'])

In [10]:
x_list = []
for x in test:
    if x == 'R.':
        x = ['Wenger', 'David R.']
    else:
        x = x.rsplit(' ', 1)
    x = x[1]
    x_list.append(x)

In [11]:
vorname = pd.DataFrame(x_list)

In [12]:
x_list = []
for x in test:
    if x == 'R.':
        x = ['Wenger', 'David R.']
    else:
        x = x.rsplit(' ', 1)
    x = x[0]
    x_list.append(x)

In [13]:
nachname = pd.DataFrame(x_list)

In [14]:
df = pd.concat([judges_list, nachname, vorname], axis=1)
df.columns = [['Name', 'Partei', 'Nachname', 'Vorname']]

In [15]:
df['Nachname Vorname'] = df['Vorname'] + ' ' + df['Nachname']

In [16]:
#Changing the one cell
df['Nachname Vorname'][26] = 'David R. Wenger'

In [17]:
df.to_csv('richter_partei.csv', index=False)

In [35]:
df

,Name,Partei,Nachname,Vorname,Nachname Vorname
0,Lang Walter,parteilos,Lang,Walter,Walter Lang
1,Bovier Gérald,SVP,Bovier,Gérald,Gérald Bovier
2,Brüschweiler Daniela,BDP,Brüschweiler,Daniela,Daniela Brüschweiler
3,Cattaneo Daniele,FDP,Cattaneo,Daniele,Daniele Cattaneo
4,Cotting-Schalch Claudia,FDP,Cotting-Schalch,Claudia,Claudia Cotting-Schalch
5,Felley Yanick,SVP,Felley,Yanick,Yanick Felley
6,Haefeli Fulvio,SVP,Haefeli,Fulvio,Fulvio Haefeli
7,Scherrer Gérard,parteilos,Scherrer,Gérard,Gérard Scherrer
8,Schürch Hans,FDP,Schürch,Hans,Hans Schürch
9,Spälti Giannakitsas Nina,SP,Spälti Giannakitsas,Nina,Nina Spälti Giannakitsas


In [36]:
df_partei_count = pd.DataFrame(df['Partei'].value_counts())

In [37]:
df_partei_count = df_partei_count.reset_index()

In [38]:
total = sum(list(df_partei_count['Partei']))

In [39]:
def per(x):
    percentage = x / total * 100
    return percentage

In [42]:
df_partei_count['percentage'] = round(df_partei_count['Partei'].apply(per))

In [43]:
df_partei_count

,index,Partei,percentage
0,SVP,7,25.0
1,SP,4,14.0
2,parteilos,4,14.0
3,Grüne,4,14.0
4,FDP,3,11.0
5,SP,2,7.0
6,GLP,2,7.0
7,CVP,1,4.0
8,BDP,1,4.0
